### Singapore Weather Prediction

Data Acquisition: Obtain the historical climate data for the variables you mentioned. The data can be obtained from the National Environment Agency or other reliable sources. Ensure the data is in a format that is compatible with your application.

In [12]:
import requests

In [17]:
surface_air_temp_monthly_mean_url = "https://data.gov.sg/api/action/datastore_search?resource_id=07654ce7-f97f-49c9-81c6-bd41beba4e96&limit=1000"
surface_air_temp_monthly_mean = requests.get(surface_air_temp_monthly_mean_url).json()
print(f'surface_air_temp_monthly_mean: {surface_air_temp_monthly_mean}')

surface_air_temp_monthly_mean: {'help': 'https://data.gov.sg/api/3/action/help_show?name=datastore_search', 'success': True, 'result': {'resource_id': '07654ce7-f97f-49c9-81c6-bd41beba4e96', 'fields': [{'type': 'int4', 'id': '_id'}, {'type': 'text', 'id': 'month'}, {'type': 'numeric', 'id': 'mean_temp'}], 'records': [{'mean_temp': '25.9', '_id': 1, 'month': '1982-01'}, {'mean_temp': '27.1', '_id': 2, 'month': '1982-02'}, {'mean_temp': '27.2', '_id': 3, 'month': '1982-03'}, {'mean_temp': '27', '_id': 4, 'month': '1982-04'}, {'mean_temp': '28', '_id': 5, 'month': '1982-05'}, {'mean_temp': '28.4', '_id': 6, 'month': '1982-06'}, {'mean_temp': '28.2', '_id': 7, 'month': '1982-07'}, {'mean_temp': '27.7', '_id': 8, 'month': '1982-08'}, {'mean_temp': '27.9', '_id': 9, 'month': '1982-09'}, {'mean_temp': '27.4', '_id': 10, 'month': '1982-10'}, {'mean_temp': '27.3', '_id': 11, 'month': '1982-11'}, {'mean_temp': '26.2', '_id': 12, 'month': '1982-12'}, {'mean_temp': '26.5', '_id': 13, 'month': '198

In [18]:
surface_air_temp_monthly_abs_extreme_max_url = "https://data.gov.sg/api/action/datastore_search?resource_id=96e66346-68bb-4ca9-b001-58bbf39e36a7&limit=1000"
surface_air_temp_monthly_abs_extreme_max = requests.get(surface_air_temp_monthly_abs_extreme_max_url).json()
print(f'surface_air_temp_monthly_abs_extreme_max: {surface_air_temp_monthly_abs_extreme_max}')

surface_air_temp_monthly_abs_extreme_max: {'help': 'https://data.gov.sg/api/3/action/help_show?name=datastore_search', 'success': True, 'result': {'resource_id': '96e66346-68bb-4ca9-b001-58bbf39e36a7', 'fields': [{'type': 'int4', 'id': '_id'}, {'type': 'text', 'id': 'month'}, {'type': 'numeric', 'id': 'max_temperature'}], 'records': [{'max_temperature': '31.9', '_id': 1, 'month': '1982-01'}, {'max_temperature': '34.3', '_id': 2, 'month': '1982-02'}, {'max_temperature': '34.3', '_id': 3, 'month': '1982-03'}, {'max_temperature': '33.2', '_id': 4, 'month': '1982-04'}, {'max_temperature': '32.9', '_id': 5, 'month': '1982-05'}, {'max_temperature': '33.3', '_id': 6, 'month': '1982-06'}, {'max_temperature': '33', '_id': 7, 'month': '1982-07'}, {'max_temperature': '33.2', '_id': 8, 'month': '1982-08'}, {'max_temperature': '33.2', '_id': 9, 'month': '1982-09'}, {'max_temperature': '33.8', '_id': 10, 'month': '1982-10'}, {'max_temperature': '34.3', '_id': 11, 'month': '1982-11'}, {'max_temperatu

Data Preprocessing: Perform necessary preprocessing steps on the data. This may include handling missing values, data cleaning, converting data types, and ensuring the data is in a suitable format for modeling.

Model Development: Implement the ARIMA model using a suitable library like statsmodels in Python. Train the model on the historical data for each variable separately. Make sure to choose appropriate orders for the AR, I, and MA components of the ARIMA model. You can use techniques like ACF and PACF plots, information criteria (AIC, BIC), and grid search to determine the optimal orders.

Model Validation: Split your data into training and validation sets. Use the training data to fit the ARIMA model and then forecast the values for the validation period. Calculate the RMSE (Root Mean Squared Error) between the predicted values and the actual values. This will serve as a measure of the model's accuracy.

Model Refinement: Iterate on the model development process by adjusting the model parameters, orders, or other settings to improve the accuracy. Consider experimenting with different variations of ARIMA models, such as SARIMA (Seasonal ARIMA) if your data exhibits seasonal patterns.

Web App Development: Create a web application using the chosen frontend framework, React. Design the user interface (UI) where users can input the number of days they want to forecast and select the variables they are interested in. Use forms or input fields to gather user input.

Backend Development: Implement the backend of your web app using Flask, a Python microframework. Set up the necessary routes and endpoints to handle user requests from the frontend. This includes receiving the user input, processing the request, and returning the forecasted values to the frontend.

Integration with ARIMA Model: Connect the frontend and backend by incorporating the ARIMA model into the Flask backend. When the user submits a forecast request, retrieve the selected variables and the number of days from the request. Use the trained ARIMA models to generate the forecasts for the specified variables and time horizon.

Displaying the Results: Once the forecasts are generated, return the results to the frontend and display them to the user. You can use visualizations, such as line charts or tables, to present the forecasted values in an intuitive and user-friendly manner.

Testing and Deployment: Test your web app thoroughly to ensure it functions correctly and provides accurate forecasts. Once you are satisfied with the app's performance, deploy it to a hosting platform or server so that it can be accessed by users.